## Dependencies

In [1]:
!pip install -q  \
    diffusers==0.24.0 \
    transformers==4.36.2 \
    accelerate==0.26.1 \
    peft==0.9.0 \
    bitsandbytes \
    opencv-python-headless

!pip install --force-reinstall torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0 --index-url https://download.pytorch.org/whl/cu126
!pip install --force-reinstall -q "numpy==1.26.4"
!pip install --force-reinstall -q "huggingface_hub==0.25.2"
!pip install -q hf_transfer
!pip install --no-cache-dir --force-reinstall pillow==11.1.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7

In [2]:
!pip install --force-reinstall --no-deps numpy==1.26.4

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [3]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)


2.9.0+cu126
0.24.0+cu126


### OOTDiffusion Repo

In [4]:
# Clone OOTDiffusion to get the model architecture definitions
import os
if not os.path.exists("OOTDiffusion"):
    !git clone https://github.com/levihsu/OOTDiffusion.git

import sys
sys.path.insert(0, os.path.abspath("OOTDiffusion"))

Cloning into 'OOTDiffusion'...
remote: Enumerating objects: 800, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 800 (delta 47), reused 27 (delta 27), pack-reused 719 (from 1)
Receiving objects: 100% (800/800), 27.31 MiB | 47.17 MiB/s, done.
Resolving deltas: 100% (149/149), done.


## Libraries

In [5]:
import argparse
import itertools
import math
import random
import json
from pathlib import Path
from PIL import Image, ImageOps
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm.auto import tqdm

# Hugging Face & Diffusers
from accelerate import Accelerator
from accelerate.logging import get_logger
from diffusers import DDPMScheduler, AutoencoderKL
from diffusers.optimization import get_scheduler
from transformers import CLIPTextModel, CLIPTokenizer, CLIPVisionModelWithProjection, AutoProcessor

# QLoRA / PEFT
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig

# OOTDiffusion Modules (from the cloned repo)
from ootd.pipelines_ootd.unet_garm_2d_condition import UNetGarm2DConditionModel
from ootd.pipelines_ootd.unet_vton_2d_condition import UNetVton2DConditionModel

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2025-12-12 12:29:51.458577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765542591.658182      

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


## Configurations

In [ ]:
class TrainConfig:

    # Assuming standard VITON-HD structure
    dataset_root = "/kaggle/input" 

    # Checkpoint saving
    save_steps = 500
    checkpoint_dir = "/kaggle/working/ootd_checkpoints"
    
    # Model Hyperparameters
    pretrained_model_name_or_path = "levihsu/OOTDiffusion"
    output_dir = "/kaggle/working/ootd_finetuned"
    seed = 42
    resolution = 512
    train_batch_size = 16
    gradient_accumulation_steps = 4
    learning_rate = 2e-5
    lr_scheduler = "constant"
    max_train_steps = 1000
    
    # QLoRA Params
    lora_r = 16
    lora_alpha = 32
    lora_dropout = 0.05
    
    gradient_checkpointing = True
    mixed_precision = "fp16"

config = TrainConfig()

## Dataset

In [ ]:
class VITONHDDataset(Dataset):
    def __init__(self, root_dir, phase="train", size=(512, 384)):
        self.root = Path(root_dir)
        self.phase = phase
        self.size = size # (H, W)
        
        self.image_dir = self.root / phase / "image"
        self.cloth_dir = self.root / phase / "cloth"
        self.parse_dir = self.root / phase / "image-parse-v3"
        
        # Get list of images
        self.image_names = sorted([p.name for p in self.image_dir.glob("*.jpg")])
        
        # Image transforms (Normalize to [-1, 1] for Diffusion)
        self.transform = transforms.Compose([
            transforms.Resize(self.size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ])
        
        # Resize cloth to match OOTD requirements (768x1024)
        self.clip_processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")

    def __len__(self):
        return len(self.image_names)

    def generate_mask(self, parse_img):
        """
        0: Background, 5: UpperClothes, 6: Dress, 7: Coat, 
        14: Left Arm, 15: Right Arm, 10: Neck, 4: Face
        """
        parse_array = np.array(parse_img)
        
        # Define indices to MASK OUT (What we want to regenerate)
        # We want to remove the old clothes and the arms (to allow repainting new sleeves)
        mask_indices = [5, 6, 7, 10, 14, 15]
        
        mask = np.zeros_like(parse_array, dtype=np.float32)
        for idx in mask_indices:
            mask += (parse_array == idx).astype(np.float32)
            
        mask = np.clip(mask, 0, 1)
        
        # Convert to PIL for resizing
        return Image.fromarray((mask * 255).astype(np.uint8))

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        
        image_path = self.image_dir / img_name
        cloth_path = self.cloth_dir / img_name
        parse_path = self.parse_dir / img_name.replace(".jpg", ".png")
        
        image = Image.open(image_path).convert("RGB")
        cloth = Image.open(cloth_path).convert("RGB")
        parse = Image.open(parse_path) # Label map
        
        mask = self.generate_mask(parse)
        
        # OOTDiffusion expects (Width, Height)
        # We resize to config.resolution (e.g., 512x384)
        target_size = (self.size[1], self.size[0]) # W, H
        
        image = image.resize(target_size, Image.BILINEAR)
        mask = mask.resize(target_size, Image.NEAREST)
        cloth = cloth.resize(target_size, Image.BILINEAR)
        
        # Normalized image [-1, 1]
        pixel_values = self.transform(image)
        
        # Normalized cloth [-1, 1] for UNetGarm
        pixel_values_cloth = self.transform(cloth)
        
        # Mask [0, 1] (1 channel)
        mask_tensor = transforms.ToTensor()(mask)
        
        # Input to UNetVton
        # We zero out the masked area in the input image
        masked_image = image.copy()
        masked_image_np = np.array(masked_image)
        mask_np = np.array(mask) / 255.0

        # Expand mask dims for broadcasting
        mask_np = np.expand_dims(mask_np, axis=2)

        # Apply mask: where mask is 1 (clothes), make image 0 (grey/black)
        masked_image_np = masked_image_np * (1 - mask_np)
        masked_image = Image.fromarray(masked_image_np.astype(np.uint8))
        pixel_values_masked = self.transform(masked_image)

        # OOTD uses CLIP vision embeddings for conditioning
        clip_input = self.clip_processor(images=cloth, return_tensors="pt").pixel_values.squeeze(0)

        return {
            "pixel_values": pixel_values,             # Target (Ground Truth)
            "pixel_values_cloth": pixel_values_cloth, # For UNetGarm
            "pixel_values_masked": pixel_values_masked, # Input to VTON UNet
            "mask": mask_tensor,                      # Conditioning Mask
            "clip_input": clip_input                  # CLIP Embeddings
        }

## Model loading Setup

In [8]:
accelerator = Accelerator(
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    mixed_precision=config.mixed_precision,
)

from huggingface_hub import snapshot_download
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
ckpt_path = "/kaggle/working/OOTDiffusion/checkpoints/ootd"
print("Downloading OOTDiffusion weights...")
os.makedirs(ckpt_path, exist_ok=True)
try:
    snapshot_download(
        repo_id="levihsu/OOTDiffusion",
        allow_patterns=["**/vae/*", "**/ootd_hd/*"],
        local_dir="/kaggle/working/OOTDiffusion/",
        local_dir_use_symlinks=False,
        resume_download=True
    )
    print("Download Successful!")
except Exception as e:
    print(f"Download failed again: {e}")
    print("Try restarting the Kaggle session and running this cell first.")

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1204: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Download Successful!


In [ ]:
# VAE
vae = AutoencoderKL.from_pretrained(f"{ckpt_path}/vae", torch_dtype=torch.float16)
vae.requires_grad_(False)

# CLIP Vision
image_encoder = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
image_encoder.requires_grad_(False)

# UNet Garm to extract garment feature
unet_garm = UNetGarm2DConditionModel.from_pretrained(
    f"{ckpt_path}/ootd_hd/checkpoint-36000",
    subfolder="unet_garm",
    torch_dtype=torch.float16
)
unet_garm.requires_grad_(False)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

unet_vton = UNetVton2DConditionModel.from_pretrained(
    f"{ckpt_path}/ootd_hd/checkpoint-36000",
    subfolder="unet_vton",
    quantization_config=bnb_config,
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

## Fine-Tuning Setup

In [ ]:
unet_vton.enable_gradient_checkpointing()

# Prepare for QLoRA
unet_vton = prepare_model_for_kbit_training(unet_vton)

lora_config = LoraConfig(
    r=config.lora_r,
    init_lora_weights="gaussian",
    lora_alpha=config.lora_alpha,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"], # Standard attention targets
    lora_dropout=config.lora_dropout,
    bias="none"
)

unet_vton = get_peft_model(unet_vton, lora_config)
unet_vton.print_trainable_parameters()

vae.to(accelerator.device)
image_encoder.to(accelerator.device)
unet_garm.to(accelerator.device)

optimizer = torch.optim.AdamW(
    unet_vton.parameters(),
    lr=config.learning_rate,
)

# Noise Scheduler (based on OOTD huggingface)
noise_scheduler = DDPMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    steps_offset=1,
    prediction_type="epsilon",
)

# Data Loader
try:
    train_dataset = VITONHDDataset(config.dataset_root, phase="train", size=(512, 384))
    train_dataloader = DataLoader(train_dataset, batch_size=config.train_batch_size, shuffle=True, num_workers=2)
except Exception as e:
    print(f"Dataset error: {e}")
    print("Creating dummy data for demonstration...")
    pass

# Prepare with Accelerator
unet_vton, optimizer, train_dataloader = accelerator.prepare(
    unet_vton, optimizer, train_dataloader
)

trainable params: 3,188,736 || all params: 862,721,220 || trainable%: 0.3696137206408346


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

## Fine-Tuning Loop

In [ ]:
print("Starting training...")
global_step = 0

# 2 epochs
for epoch in range(2):
    unet_vton.train()
    for step, batch in enumerate(train_dataloader):
        with accelerator.accumulate(unet_vton):
            # Convert images to latents
            latents = vae.encode(batch["pixel_values"].to(dtype=torch.float16)).latent_dist.sample()
            latents = latents * vae.config.scaling_factor
            
            # Encode masked image (conditioning)
            masked_latents = vae.encode(batch["pixel_values_masked"].to(dtype=torch.float16)).latent_dist.sample()
            masked_latents = masked_latents * vae.config.scaling_factor
            
            # Resize mask to latent size
            mask = F.interpolate(batch["mask"], size=(latents.shape[2], latents.shape[3]), mode="nearest")
            
            # Sample noise
            noise = torch.randn_like(latents)
            bsz = latents.shape[0]
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
            
            # Add noise to latents (Forward Diffusion)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
            
            # Conditioning
            # CLIP embeddings
            with torch.no_grad():
                clip_embeds = image_encoder(batch["clip_input"].to(dtype=torch.float16)).image_embeds
                clip_embeds = clip_embeds.unsqueeze(1) # [B, 1, 1024]
                
            # UNetGarm features
            with torch.no_grad():
                cloth_latents = vae.encode(batch["pixel_values_cloth"].to(dtype=torch.float16)).latent_dist.sample()
                cloth_latents = cloth_latents * vae.config.scaling_factor
                
                # OOTD inference runs unet_garm with timestep 0 (clean image), not random noise steps
                zero_timesteps = torch.zeros((cloth_latents.shape[0],), device=cloth_latents.device, dtype=torch.long)
                
                _, garm_features = unet_garm(
                    cloth_latents, 
                    zero_timesteps, 
                    encoder_hidden_states=clip_embeds, 
                    return_dict=False
                )

            # Forward Pass (UNet VTON)
            # Concatenate noisy latents and masked_latents
            # OOTD expects 8 channels (4 noisy + 4 masked). 
            unet_input = torch.cat([noisy_latents, masked_latents], dim=1)
            
            # Predict noise
            noise_pred = unet_vton(
                sample=unet_input, 
                spatial_attn_inputs=garm_features,    # The spatial features from UNetGarm
                timestep=timesteps, 
                encoder_hidden_states=clip_embeds,    # Text/CLIP condition
            ).sample

            loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")
            
            accelerator.backward(loss)
            optimizer.step()
            optimizer.zero_grad()
        
        if accelerator.sync_gradients:
            global_step += 1
            if global_step % 10 == 0:
                print(f"Step {global_step}, Loss: {loss.item()}")

            if global_step % config.save_steps == 0:
                save_path = os.path.join(config.checkpoint_dir, f"checkpoint-{global_step}")
                os.makedirs(save_path, exist_ok=True)
                print(f"Saving checkpoint to {save_path}...")
                
                # Unwrap model (remove accelerator/DDP wrappers)
                unwrapped_model = accelerator.unwrap_model(unet_vton)
                
                # Save only the LoRA adapters
                unwrapped_model.save_pretrained(save_path)
                print(f"Save completed!")
            
            if global_step >= config.max_train_steps:
                break
    
    if global_step >= config.max_train_steps:
        break

Starting training...
Step 10, Loss: 0.16111576557159424
Step 20, Loss: 0.09066855907440186
Step 30, Loss: 0.11368381977081299
Step 40, Loss: 0.08677985519170761
Step 50, Loss: 0.051965873688459396
Step 60, Loss: 0.11663501709699631
Step 70, Loss: 0.13146626949310303
Step 80, Loss: 0.056867167353630066
Step 90, Loss: 0.23047363758087158
Step 100, Loss: 0.125235915184021
Step 110, Loss: 0.11220284551382065
Step 120, Loss: 0.07533510029315948
Step 130, Loss: 0.13184428215026855
Step 140, Loss: 0.06976653635501862
Step 150, Loss: 0.1007581576704979
Step 160, Loss: 0.0764094740152359
Step 170, Loss: 0.11198130249977112
Step 180, Loss: 0.06985907256603241
Step 190, Loss: 0.10234558582305908
Step 200, Loss: 0.10045932978391647
Step 210, Loss: 0.0687512457370758
Step 220, Loss: 0.06297695636749268
Step 230, Loss: 0.03222188353538513
Step 240, Loss: 0.08185449242591858
Step 250, Loss: 0.11369757354259491
Step 260, Loss: 0.08543567359447479
Step 270, Loss: 0.14005617797374725
Step 280, Loss: 0.1

In [13]:
print("Saving LoRA weights...")
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(unet_vton)
unwrapped_model.save_pretrained(config.output_dir)
print(f"Saved to {config.output_dir}")

Saving LoRA weights...
Saved to /kaggle/working/ootd_finetuned
